<a href="https://colab.research.google.com/github/elifgularslan/AnimalClassificationWithCNN/blob/main/AnimalClassificationWithCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!pip install kaggle


### ☁️ 0. Dataset Download & Setup (Veri Seti İndirme ve Kurulum)

**English:**
In this step, we fetch the **Animals-10** dataset directly from Kaggle into the Google Colab environment.
* **Kaggle API:** We use the `!kaggle datasets download` command to pull the dataset quickly without manual uploading.
* **Unzip:** The dataset comes in a compressed `.zip` format. We use `!unzip` to extract the images into a specific folder named `animals10` for easy access.

**Türkçe:**
Bu adımda, **Animals-10** veri setini doğrudan Kaggle üzerinden Google Colab ortamına çekiyoruz.
* **Kaggle API:** Veri setini elle yüklemekle uğraşmadan, hızlıca indirmek için `!kaggle datasets download` komutunu kullanıyoruz.
* **Unzip (Zip'ten Çıkarma):** Veri seti sıkıştırılmış `.zip` formatında gelir. Resimleri erişilebilir hale getirmek ve düzenli tutmak için `!unzip` komutuyla `animals10` klasörüne çıkartıyoruz.

In [ ]:
!kaggle datasets download -d alessiocorrado99/animals10
!unzip animals10.zip -d animals10


### 📚 1. Libraries & Setup (Kütüphaneler ve Kurulum)

**English:**
In this section, we import the necessary libraries.
* **TensorFlow & Keras:** The core deep learning framework used to build and train the model.
* **Matplotlib:** Used for visualizing the data, training graphs (accuracy/loss), and prediction results.
* **OpenCV (cv2):** Used for advanced image processing, specifically for reading images from disk and converting color spaces (BGR to RGB).
* **Numpy:** Fundamental package for scientific computing and array manipulation.
* **GC (Garbage Collector):** Used to manually clean up RAM to prevent session crashes in Google Colab.

**Türkçe:**
Bu bölümde gerekli kütüphaneleri içe aktarıyoruz.
* **TensorFlow & Keras:** Modeli kurmak ve eğitmek için kullandığımız temel derin öğrenme çatısı.
* **Matplotlib:** Veriyi, eğitim grafiklerini (doğruluk/kayıp) ve tahmin sonuçlarını görselleştirmek için kullanılır.
* **OpenCV (cv2):** Gelişmiş görüntü işleme için kullanılır; özellikle diskten resim okuma ve renk uzayı dönüşümleri (BGR'den RGB'ye) yapmak için gereklidir.
* **Numpy:** Bilimsel hesaplamalar ve dizi (array) işlemleri için temel paket.
* **GC (Çöp Toplayıcı):** Google Colab'de oturum çökmesini önlemek amacıyla RAM'i manuel olarak temizlemek için kullanılır

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.layers import RandomRotation, RandomZoom, RandomFlip, RandomContrast, GaussianNoise
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import gc # garbage collector

# TR: RAM şişmesini önlemek için çöp toplayıcıyı çalıştırıyoruz.
# EN: Running garbage collector to prevent RAM overflow.
gc.collect()
tf.keras.backend.clear_session()



### 📥 2. Data Loading & Preprocessing (Veri Yükleme ve Ön İşleme)

**English:**
We load the **Animals-10** dataset using `image_dataset_from_directory`.
* **Image Size:** We resize all images to `224x224` pixels to match standard CNN input requirements.
* **Split:** The data is split into **Training (80%)** and **Validation (20%)** sets.
* **Optimization:** We use `prefetch` instead of `cache` to stream data efficiently without overloading the RAM.

**Türkçe:**
**Animals-10** veri setini `image_dataset_from_directory` fonksiyonu ile yüklüyoruz.
* **Görüntü Boyutu:** Standart CNN giriş gereksinimlerine uyması için tüm resimleri `224x224` piksel boyutuna getiriyoruz.
* **Ayırma:** Veri seti **Eğitim (%80)** ve **Doğrulama (%20)** olarak ikiye ayrıldı.
* **Optimizasyon:** RAM'i şişirmeden veriyi verimli bir şekilde akıtmak için `cache` yerine `prefetch` yöntemini kullanıyoruz.

In [ ]:
data_dir = "/content/animals10/raw-img"

# TR: Resim boyutu. Model 224x224 piksel bekler.
# EN: Image size. The model expects 224x224 pixels.
IMG_SIZE = 224

# TR: Bir seferde işlenen resim sayısı.
# EN: Number of images processed at once
BATCH_SIZE = 32


print("Does the folder exist?", os.path.exists(data_dir))
print("Contents:", os.listdir(data_dir))




# TR: Eğitim veri setini oluşturuyoruz (%80)
 # EN: Creating the training dataset (80%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,    # TR: Verinin %20'sini test için ayır. / EN: Split 20% for validation.
    subset="training",       # TR: Bu kısım eğitim için. / EN: This subset is for training.
    seed=42,                 # TR: Rastgelelik sabiti (her seferinde aynı ayrımı yapar). / EN: Random seed for reproducibility.
    label_mode='int',        # TR: Etiketler tamsayı (0, 1, 2...) olarak gelir. / EN: Labels are returned as integers.
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)


# TR: Doğrulama (Test) veri setini oluşturuyoruz (%20)
    # EN: Creating the validation dataset (20%)


val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes ({num_classes} total):", class_names)


# TR: Performans ayarı. Veriyi önbelleğe almadan (cache yok) sıradaki veriyi hazırlar.
# EN: Performance tuning. Prefetches the next batch without caching (saves RAM)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

### 🔍 3. Exploratory Data Analysis (Keşifçi Veri Analizi)

**English:**
Before training, it is crucial to visualize the dataset to ensure data integrity.
In this block, we randomly select and display one sample image from each of the 10 classes. This helps us verify that the labels are correct and the images are loaded properly.

**Türkçe:**
Eğitime başlamadan önce, veri bütünlüğünü doğrulamak için veri setini görselleştirmek çok önemlidir.
Bu blokta, 10 sınıfın her birinden rastgele birer örnek seçip ekrana basıyoruz. Bu işlem, etiketlerin doğruluğunu ve resimlerin düzgün yüklendiğini teyit etmemizi sağlar.

In [ ]:

import random

# ==========================================
# 3. DATA EXPLORATION (VERİ KEŞFİ)
# ==========================================
# TR: Her sınıftan rastgele bir örnek göstererek veri setini tanıyalım.
# EN: Let's explore the dataset by showing one random sample from each class.


data_dir = "/content/animals10/raw-img"

# TR: Tüm sınıfları bul ve sırala
# EN: Find and sort class directories
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
classes.sort()

print(f"Number of classes: {len(classes)}")
print(f"Classes: {classes}")

# TR: 10 Sınıf için 2 satır 5 sütunluk tablo ayarla
# EN: Set up a table with 2 rows and 5 columns for the 10 classes
plt.figure(figsize=(15, 6))

for i, class_name in enumerate(classes):
    class_path = os.path.join(data_dir, class_name)

    # TR:Klasördeki tüm resimleri listele
    # EN: List valid image files in the directory.
    files = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    # TR: Rastgele bir dosya seç. / EN: Pick a random file.
    if len(files) > 0:
        random_file = random.choice(files)
        img_path = os.path.join(class_path, random_file)

        # Resmi oku ve renklendir
        # Read and color the image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Tabloya yerleştir (i+1 çünkü subplot 1'den başlar)
        # Place into the table (i+1 because subplot starts from 1)

        plt.subplot(2, 5, i + 1)
        plt.imshow(img)
        plt.title(class_name.capitalize(), fontsize=12, fontweight='bold') # TR:Başlık (Kedi, Köpek vs.) # EN:Title (Cat, Dog etc.)
        plt.axis("off") # TR:Çerçeveyi kaldır # EN:Remove frame

plt.tight_layout()
plt.suptitle("Random Samples from Animals-10 Dataset", fontsize=16, y=1.02)
plt.show()

### 🧠 4. Model Architecture (Model Mimarisi)

**English:**
We are building a **Custom CNN (Convolutional Neural Network)** from scratch.
* **Robustness:** We add `GaussianNoise` and Data Augmentation (Flip, Rotation, Zoom) layers to make the model resistant to image noise and prevent overfitting.
* **Rescaling:** Normalizes pixel values from `0-255` to `0-1` range for faster convergence.
* **Feature Extraction:** We use 3 blocks of `Conv2D` (Convolution) and `MaxPooling2D` layers to detect edges, textures, and shapes.
* **Classifier:** A fully connected `Dense` layer with `Dropout(0.3)` is used to classify the features into 10 categories.

**Türkçe:**
Sıfırdan (from scratch) bir **Özel CNN (Evrişimli Sinir Ağı)** inşa ediyoruz.
* **Dayanıklılık (Robustness):** Modeli görüntü gürültüsüne karşı dirençli hale getirmek ve aşırı öğrenmeyi (overfitting) önlemek için `GaussianNoise` ve Veri Artırma (Çevirme, Döndürme, Yakınlaştırma) katmanları ekliyoruz.
* **Ölçeklendirme:** Piksel değerlerini `0-255` aralığından `0-1` aralığına çekerek eğitimin daha hızlı yakınsamasını sağlar.
* **Özellik Çıkarımı:** Kenarları, dokuları ve şekilleri algılamak için 3 bloklu `Conv2D` ve `MaxPooling2D` katmanları kullanıyoruz.
* **Sınıflandırıcı:** Özellikleri 10 kategoriye ayırmak için `Dropout(0.3)` içeren tam bağlantılı bir `Dense` katman kullanıyoruz.

In [ ]:
# ==========================================
# 4. MODEL ARCHITECTURE (MODEL MİMARİSİ)
# ==========================================

# TR: Veri Artırma Katmanı. Ezberlemeyi önlemek için resimleri değiştirir.
# EN: Data Augmentation Layer. Modifies images to prevent overfitting.
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),   # TR: Yatay çevir / EN: Horizontal flip
    layers.RandomRotation(0.1),        # TR:%10 Döndür / EN: Rotate 10%
    layers.RandomZoom(0.1),            # TR: %10 Yakınlaştır / EN: Zoom 10%
    layers.RandomContrast(0.2),        # TR: Kontrast değiştir / EN: Adjust contrast
    layers.RandomBrightness(0.2),
    layers.GaussianNoise(0.02),        # TR: Hafif gürültü ekle / EN: Add slight noise
], name="augmentation_layer")



In [ ]:
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# TR: Önce veri artırmayı uygula. / EN: Apply data augmentation first.
x = data_augmentation(inputs)

# TR: Normalizasyon. Piksel değerlerini 0-255 arasından 0-1 arasına çeker.
# EN: Normalization. Scales pixel values from 0-255 to 0-1 range.
x = layers.Rescaling(1/255)(x)

# --- CNN BLOCKS (CNN BLOKLARI) ---
# TR: Özellik Çıkarma (Kenarlar, şekiller vb.)
# EN: Feature Extraction (Edges, shapes, etc.)

# Block 1
x = layers.Conv2D(32, 3, padding='same', activation='relu')(x) # 32 Filters
x = layers.MaxPooling2D()(x)    # TR: Boyutu yarıya indir / EN: Reduce size by half

# Block 2
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.MaxPooling2D()(x)

# Block 3
x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
x = layers.MaxPooling2D()(x)


# --- CLASSIFICATION  (SINIFLANDIRMA ) ---
x = layers.Flatten()(x)# TR: Düz vektöre çevir / EN: Flatten to vector

# TR: Tam Bağlantılı Katman / EN: Fully Connected Layer
x = layers.Dense(256, activation='relu')(x)

# TR: Unutma Katmanı. Nöronların %30'sini rastgele kapatır (Overfitting önlemi).
# EN: Dropout Layer. Randomly turns off 30% of neurons (Prevents overfitting).
x = layers.Dropout(0.3)(x)


# TR: Çıkış Katmanı. Her sınıf için bir olasılık üretir.
# EN: Output Layer. Produces a probability for each class.
outputs = layers.Dense(num_classes, activation='softmax')(x)


# TR: Modeli birleştir ve derle.
# EN: Construct and compile the model.
model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


### 🚀 5. Training with Callbacks (Callback ile Eğitim)

**English:**
To optimize the training process, we use intelligent callbacks:
* **EarlyStopping:** Monitors `val_loss`. If the model stops improving for 5 epochs, training is stopped automatically to save time and prevent overfitting. It restores the best weights found.
* **ReduceLROnPlateau:** Monitors `val_loss`. If the loss plateaus (stops decreasing), it reduces the learning rate by a factor of `0.2` to allow fine-tuning.

**Türkçe:**
Eğitim sürecini optimize etmek için akıllı "geri çağırma" (callback) mekanizmaları kullanıyoruz:
* **EarlyStopping (Erken Durdurma):** `val_loss` değerini izler. Eğer model 5 epoch boyunca gelişmezse, zaman kazanmak ve aşırı öğrenmeyi önlemek için eğitimi otomatik olarak durdurur. Bulunan en iyi ağırlıkları geri yükler.
* **ReduceLROnPlateau:** `val_loss` değerini izler. Eğer kayıp değeri sabitlenirse (düşmeyi bırakırsa), öğrenme hızını `0.2` kat düşürerek modelin daha hassas öğrenmesini (fine-tuning) sağlar.

In [ ]:
# ==========================================
# 5. CALLBACKS (GERİ ÇAĞIRMALAR)
# ==========================================


# TR: Erken Durdurma. Eğer model gelişmeyi durdurursa eğitimi keser.
# EN: Early Stopping. Stops training if the model stops improving.
early_stop = EarlyStopping(
    monitor='val_loss',       # Test hatasını takip et # Track validation loss
    patience=5,               # 5 epoch boyunca iyileşme olmazsa dur# Stop if no improvement for 5 epochs
    restore_best_weights=True,# En iyi sonucu hafızada tut (ÇOK ÖNEMLİ)# Keep the best model weights (VERY IMPORTANT)
    verbose=1
)

# TR: Öğrenme Hızı Azaltıcı. Model tıkanırsa adımları küçültür.
# EN: Learning Rate Reducer. Reduces step size if model gets stuck.
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,               # Tıkanırsa hızı 5 kat düşür  # If plateau, reduce learning rate by 5x
    patience=2,               # 2 epoch sabret, düzelmezse hızı düşür # Wait 2 epochs, then reduce LR if no improvement
    min_lr=1e-6,
    verbose=1
)



# ==========================================
# 6. TRAINING (EĞİTİM)
# ==========================================

print("\n Starting Training...")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=[early_stop, reduce_lr]
)


# ==========================================
# 7. VISUALIZATION & SAVING (GÖRSELLEŞTİRME VE KAYIT)
# ==========================================
import matplotlib.pyplot as plt

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

plt.figure(figsize=(14,5))

# Doğruluk Grafiği # Accuracy Plot
plt.subplot(1,2,1)
plt.plot(acc, label="Train Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.title("Accuracy")
plt.legend()
plt.grid(True)

# Kayıp Grafiği  # Loss Plot
plt.subplot(1,2,2)
plt.plot(loss, label="Train Loss")
plt.plot(val_loss, label="Validation Loss")
plt.title("Loss")
plt.legend()
plt.grid(True)

plt.show()

# TR: Modeli kaydet / EN: Save the model
model_name = "animals10_Model.keras"
model.save(model_name)
print(f"Model saved successfully: {model_name}")

### 📊 6. Evaluation & Prediction (Değerlendirme ve Tahmin)

**English:**
In the final step, we evaluate the model's performance:
* **Learning Curves:** We plot Accuracy and Loss graphs to analyze the training progress and check for overfitting/underfitting.
* **Testing:** We select a random image from the validation set that the model has never seen before and perform a prediction to visualize the result in real-time.

**Türkçe:**
Son adımda, modelin performansını değerlendiriyoruz:
* **Öğrenme Eğrileri:** Eğitimin ilerleyişini analiz etmek ve aşırı öğrenme (overfitting) durumunu kontrol etmek için Doğruluk (Accuracy) ve Kayıp (Loss) grafiklerini çizdiriyoruz.
* **Test Etme:** Doğrulama setinden, modelin daha önce hiç görmediği rastgele bir resim seçiyoruz ve sonucu gerçek zamanlı olarak görmek için bir tahmin gerçekleştiriyoruz.

In [ ]:
import random


# ==========================================
# 8. PREDICTION TEST (TAHMİN TESTİ)
# ==========================================
print("\n Testing with a Random Image...")

def predict_image_corrected(path):
   # TR: Resmi yükle ve modelin anlayacağı boyuta (224x224) getir.
    # EN: Load the image and resize to target size (224x224).
    img = tf.keras.utils.load_img(path, target_size=(IMG_SIZE, IMG_SIZE))

       # TR: Resmi diziye (array) çevir.
    # EN: Convert image to array.
    arr = tf.keras.utils.img_to_array(img)

    # TR: Batch boyutu ekle (Model tek resim değil, resim grubu bekler).
    # EN: Add batch dimension (Model expects a batch, not a single image).
    arr = np.expand_dims(arr, axis=0)

    # TR: Tahmin yap.
    # EN: Make prediction.
    pred = model.predict(arr)[0]
    idx = np.argmax(pred)
    prob = pred[idx]


    # TR: Gerçek sınıf ismini dosya yolundan al (DÜZELTİLEN KISIM BURASI)
    # EN: Get actual class name from file path (FIXED HERE)
    # random_path yerine path kullanıyoruz!
    actual_class = path.split('/')[-2]

    # TR: Sonucu Göster.
    # EN: Show Result.
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.axis('off')

    # TR: Başlık: Gerçek Sınıf vs Tahmin Edilen Sınıf
    # EN: Title: Actual Class vs Predicted Class
    plt.title(f"Actual: {actual_class}\nPredicted: {class_names[idx]} ({prob*100:.2f}%)")


    plt.show()

# --- MAIN EXECUTION (ANA ÇALIŞTIRMA) ---

import glob

val_dir = "/content/animals10/raw-img"

 # TR: Klasördeki tüm resim formatlarını (.jpg, .png, .jpeg) bul.
# EN: Find all image formats (.jpg, .png, .jpeg) in the directory.
all_images = glob.glob(val_dir + "/*/*.jpg") + glob.glob(val_dir + "/*/*.jpeg") + glob.glob(val_dir + "/*/*.png")

if len(all_images) > 0:
    # TR: Rastgele bir resim seç.
    # EN: Choose a random image.
    random_image_path = random.choice(all_images)
    print(f"Selected Image :{random_image_path}")

    # TR: Fonksiyonu çağır.
    # EN: Call the function.
    predict_image_corrected(random_image_path)
else:
    print(" No images found! (Resim bulunamadı!")